In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics

from src.accelerometer import plot_fourier_transformation, plot_acceleration, plot_acceleration_subplots, plot_feature_columns, accelerometer_feature_engineering
from src.kmeans import kmeans
from src.principal_component_analysis import principal_component_analysis, plot_principal_component_analysis
from src.decision_tree import decision_tree

In [ ]:
slow_move_df = pd.read_csv('slow_moves.csv')
slow_move_df = slow_move_df[slow_move_df['age_group']!=0]

In [ ]:
slow_move_df.head(5)

In [ ]:
slow_move_df.describe()

In [ ]:
single_session_df = slow_move_df[slow_move_df['uuid'] == slow_move_df['uuid'].unique()[0]]

In [ ]:
plot_acceleration(single_session_df, 'test session')

In [ ]:
plot_fourier_transformation(single_session_df, 'test session')

In [ ]:
df_std = slow_move_df.groupby(['uuid'])['x', 'y', 'z', 'mag'].agg('std')
df_std.boxplot()

In [ ]:
plot_acceleration_subplots(slow_move_df)

In [ ]:
slow_move_processed_df = accelerometer_feature_engineering(slow_move_df)

In [ ]:
slow_move_processed_df.head()

In [ ]:
slow_move_processed_df.corr()

SEM is closely related to std -> therefore we drop it

In [ ]:
slow_move_processed_df = slow_move_processed_df.drop(columns=['x_sem', 'y_sem', 'z_sem', 'mag_sem'])
slow_move_processed_df.corr()

In [ ]:
feature_keys = ['x_std','y_std','z_std','mag_std','x_mean','y_mean','z_mean','mag_mean','x_peaks','y_peaks','z_peaks','mag_peaks','x_snr','y_snr','z_snr','mag_snr', 'x_sal','y_sal','z_sal','mag_sal','duration']
class_key = ['age_group']

In [ ]:
plot_feature_columns(slow_move_processed_df,class_key,'std')
plot_feature_columns(slow_move_processed_df,class_key,'mean')
plot_feature_columns(slow_move_processed_df,class_key,'peaks')
plot_feature_columns(slow_move_processed_df,class_key,'snr')
plot_feature_columns(slow_move_processed_df,class_key,'sal')

We can see that there is no significant deviation between the two age groups in the following features:
- y_mean
- mag_mean
- z_snr
- mag_snr
- z_sal
- mag_sal

In [ ]:
slow_move_processed_df = slow_move_processed_df.reset_index(drop=False)
train_df, test_df = train_test_split(slow_move_processed_df, test_size=0.10)

In [ ]:
feature_keys = ['x_std','z_std', 'z_std','mag_std','x_mean','y_mean','x_peaks','y_peaks','z_peaks','mag_peaks','x_snr','y_snr','x_sal','y_sal']

# Apply Kmeans Clustering

In [ ]:
kmeans_model = kmeans(train_df, feature_keys)
kmeans_predicitons = kmeans_model.predict(test_df[feature_keys])
predictions_df = pd.DataFrame()
predictions_df['age_group'] = test_df['age_group']
predictions_df['cluster'] = kmeans_predicitons
predictions_df

# Principal Component Analysis

In [ ]:
principal_components_df = principal_component_analysis(slow_move_processed_df, feature_keys)

In [ ]:
plot_principal_component_analysis(slow_move_processed_df, principal_components_df, 'age_group', [30,50])

# Decision Tree

In [ ]:
dt = decision_tree(train_df, feature_keys)

In [ ]:
dt_predictions = dt.predict(test_df[feature_keys])
metrics.accuracy_score(test_df['age_group'], dt_predictions)